In [1]:
from scipy import stats
import numpy as np
from src.boot import boot_sample

In [ ]:
def bootstrap_ATT(B, g, delta, your_data, unit_id, time):
    """
    B: Number of bootstrap samples
    g: Parameter g
    delta: Parameter delta
    your_data: The data you are working with, format depends on your needs
    unit_id: Column name for unit IDs
    time: Column name for time
    """
    ATT_bootstrap_results = []
    R_bootstrap_results = []
    param_cov_matrices = []  # To store covariance matrices from each bootstrap
    
    for b in range(B):
        # Step 1: Use boot_sample to draw the bootstrap sample
        bs_data = boot_sample(your_data, unit_id, time)
        
        # Step 2: Modularized computation of ATT_hat_star and its covariance matrix
        ATT_hat_star, cov_matrix = compute_ATT(bs_data)  # Make compute_ATT return covariance matrix
        param_cov_matrices.append(cov_matrix)
        
        # Forming R_hat_star
        R_hat_star = np.sqrt(n) * (ATT_hat_star - ATT_hat)
        
        # Storing results
        ATT_bootstrap_results.append(ATT_hat_star)
        R_bootstrap_results.append(R_hat_star)
        
    # Step 4: Bootstrap estimator of Sigma^(1/2)
    q_75 = np.percentile(R_bootstrap_results, 75)
    q_25 = np.percentile(R_bootstrap_results, 25)
    z_75 = stats.norm.ppf(0.75)
    z_25 = stats.norm.ppf(0.25)
    sigma_half_hat = (q_75 - q_25) / (z_75 - z_25)
    
    # Step 5 and 6: Compute t-tests and empirical quantiles
    t_tests = [np.abs(R) / sigma_half_hat for R in R_bootstrap_results]
    c_1_alpha = np.percentile(t_tests, 95)  # Assuming alpha = 0.05
    
    # Step 7: Construct confidence bands (Extendable to simultaneous intervals)
    ATT_hat_nev_dr, _ = compute_ATT(None, your_data)  
    confidence_band = [ATT_hat_nev_dr + c_1_alpha * sigma_half_hat / np.sqrt(n),
                       ATT_hat_nev_dr - c_1_alpha * sigma_half_hat / np.sqrt(n)]
    
    # Average parameter covariance matrix over all bootstraps
    avg_param_cov_matrix = np.mean(param_cov_matrices, axis=0)
    
    # Standard errors from the diagonal of the average covariance matrix
    param_std_errors = np.sqrt(np.diag(avg_param_cov_matrix))
    
    return confidence_band, param_std_errors